# Thinking1:关联规则中的支持度、置信度和提升度代表的什么，如何计算

    
   |     | 概念  | 公式  |
   |  ----  | ----  |  ----  |
   | 支持度  | 商品在指定交易集合中出现的次数 |商品次数/总交易次数 |
   | 置信度  | 在商品A出现的情况下，商品b出现的概率，也就是商品B在有商品A出现的交易中出现的次数 |  商品AB同时出现的次数/商品A出现的次数 |
   | 提升度  | 商品A和商品B同时出现时候，商品B对商品A的支持程度 |支持度/置信度 |
    

# Thinking2 关联规则与协同过滤的区别
    
     关联规则：是基于交易行为对物体固有的内部关系进行发掘，找出其频繁项集
     协同过滤：是基于用户的历史行为习惯进行分析，挖掘用户喜好来进行推荐

# Thinking3 为什么我们需要多种推荐算法

   各种推荐算法都有其优点以及适用场景，比如在电商系统中，对于一些冷启动的用户或这商品，可以通过关联规则方式进行前期推荐，等到有一定的用户行为或用户对商品的操作反馈信息后，可以使用协同过滤的方式推荐。通过关联规则可以发掘一些整体上的宏观规律性质的信息，协同过滤更具有个性化的需求


# Thinking4 关联规则中的最小支持度、最小置信度该如何确定
 
 通过经验数据来设置，比如交易数据量很大，此时每个商品在总商品中出现的次数都很低，如果设置的最小支持度或者最小置信度过大，则很可能筛选不出任何结果
 也可以将每个商品的支持度，置信度计算出来，从大到小排序，然后卡定符合业务需求的位置所对应的指出度或置信度为最小支持度或者最小置信度


# Thinking5  都有哪些常见的回归分析方法，评价指标是什么

  常见的回归分析方法有：一元线性回归，多元线性回归 非线性回归
  评价指标是通过损失函数来评价的，有mse(更容易收敛)  MAE(可解释性更强) RMSE(既收敛又兼具可解释性)  R方值


# Action1  针对MarketBasket数据集进行购物篮分析（频繁项集及关联规则挖掘）[kaggle](https://www.kaggle.com/dragonheir/basket-optimisation)
         

In [3]:
'''
通过mlxtend的apriori方式对数据进行分析
'''

import pandas as pd
import fptools as fp
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
# 读取数据
datas=pd.read_csv('data/Market_Basket_Optimisation.csv',header=None)
# 通过迭代器方式对pd读取出来的数据转成list数组，并且出去其中的nan
itemsets = [[y for y in x if pd.notna(y)] for x in datas.values.tolist()]

# 转换为算法可接受模型（布尔值）one-hot操作
te = TransactionEncoder()
df_tf = te.fit_transform(itemsets)
df = pd.DataFrame(df_tf, columns=te.columns_)

# 设置支持度求频繁项集
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)
# 求关联规则,设置最小置信度为0.15
rules = association_rules(frequent_itemsets,metric='confidence',  min_threshold=0.15)
# 删除提升度小于1.2的关联项
rules = rules.drop(rules[rules.lift < 1.2].index)
# 对提升度由大到小排序
rules = rules.sort_values(by="lift",ascending=False) 
# # 设置标题索引并打印结果
rules.rename(columns={'antecedents': 'from', 'consequents': 'to', 'support': 'sup', 'confidence': 'conf'}, inplace=True)
rules = rules[['from', 'to', 'sup', 'conf', 'lift']]
print(rules)





                   from                   to       sup      conf      lift
52          (spaghetti)        (ground beef)  0.039195  0.225115  2.291162
53        (ground beef)          (spaghetti)  0.039195  0.398915  2.291162
67          (olive oil)          (spaghetti)  0.022930  0.348178  1.999758
62               (soup)      (mineral water)  0.023064  0.456464  1.914955
41               (milk)  (frozen vegetables)  0.023597  0.182099  1.910382
40  (frozen vegetables)               (milk)  0.023597  0.247552  1.910382
1             (burgers)               (eggs)  0.028796  0.330275  1.837830
0                (eggs)            (burgers)  0.028796  0.160237  1.837830
59          (olive oil)      (mineral water)  0.027596  0.419028  1.757904
70           (tomatoes)          (spaghetti)  0.020931  0.306043  1.757755
50      (mineral water)        (ground beef)  0.040928  0.171700  1.747522
51        (ground beef)      (mineral water)  0.040928  0.416554  1.747522
49        (ground beef)  

In [5]:
import pandas as pd
import fptools as fp
import collections

# 读取数据
# datas=pd.read_csv('./Market_Basket_Optimisation.csv',header=None)
# # 通过迭代器方式对pd读取出来的数据转成list数组，并且出去其中的nan
# itemsets = [[y for y in x if pd.notna(y)] for x in datas.values.tolist()]
# itemDits = {}

'''
整体流程
加载数据，并且对数据清洗，去除nan数据

计算订单中每个商品出现的次数，以及最小支持度，并且对其由大到小排序

根据最小支持度阈值获取符合条件的商品列表

根据商品列表对每一个订单所对应的商品进行排序

根据排序订单项构建fp-growth树

从最后一个向上遍历得到条件模式基对应的平凡项集

计算对应的支持度，置信度，提升度

'''
# 整体流程
# 加载数据，并且对数据清洗，去除nan数据
is_find_min_support = False


def loadUseAbleData():
    datas = pd.read_csv('data/Market_Basket_Optimisation.csv', header=None)
    # 通过迭代器方式对pd读取出来的数据转成list数组，并且出去其中的nan
    return [[y for y in x if pd.notna(y)] for x in datas.values.tolist()]


def directItem(itemList, minSup, minItemCount):
    """
    把商品标签化，并且映射为字典，然后计算订单中每个商品出现的次数，以及最小支持度，并且对其由大到小排序
    :param itemList: 商品集
    :param minSup: 最小支持度
    :param minItemCount: 所需要的商品数量集合，用于根据业务筛选推荐的商品总数 当is_find_min_support=true时候 该项比较重要
    :return:
    """
    itemDits = {}
    # 遍历数组
    for items in itemList:
        for item in items:
            if item not in itemDits.keys():
                itemDits[item] = 1
            else:
                itemDits[item] = itemDits[item] + 1
    # 当为获取最小支持度时候
    if is_find_min_support:
        itemDitsList = sorted(itemDits.items(), key=lambda item: item[1], reverse=True)[:minItemCount]
        print('第{0}个商品所对应的支持度为{1},对应的商品出现的频次为：{2},总订单数{3}'.format(minItemCount, itemDitsList[-1][1] / len(itemList),
                                                                  itemDitsList[-1][1], len(itemList)))
        print(itemDitsList)
        return dict(itemDitsList)
    minCount = minSup * len(itemList)
    itemDits = {k: v for k, v in itemDits.items() if v > minCount}
    return sorted(itemDits.items(), key=lambda item: item[1], reverse=True)


def sortedAndFiltter(itemList, orgItemList):
    """
    对原始数据根据 itemsList排序，并将不存在于itemsList中对商品过滤掉
    :param itemList:
    :param itemsList: 目标商品列表
    :param orgItemList: 原始订单所对应的商品列表
    """
    itemSet = set(k[0] for k in itemList)
    itemDict = dict(itemsList)
    for index in range(len(orgItemList)):
        orgItemList[index] = list(set(orgItemList[index]).intersection(itemSet))
        orgItemList[index] = sorted(orgItemList[index], key=lambda s: itemDict[s], reverse=True)
    return itemDict, orgItemList


class FpNode(object):
    def __init__(self, code, parent=None):
        """
        树的基本结构单元
        """
        self.code = code
        self.child = collections.defaultdict()
        self.parent = parent
        # 说明是初始化的
        if parent is None:
            self.count = 0
            return
        self.count = 1
        parent.child.setdefault(code, self)
        return

    def add_count(self):
        self.count += 1
        return


class tree(object):
    """
     构建的fp-growth有且只有一个根节点，元素值为None
    """

    def __init__(self, itemDict, transaction_count):
        self.transaction_count = transaction_count
        self.root = FpNode(None, None)
        # 用于快速检索到所有同一个code的所有树叶信息 格式为 code：[]
        self.fpNodeDict = collections.defaultdict()
        self.itemDict = itemDict
        return

    def insert_node(self, transaction_items):
        """
        根据指定订单中的所有商品信息构建树的一条路径
        """
        parent = self.root
        for item in transaction_items:
            code = item  # hash(item)
            currentNode = parent.child.get(code)
            # 如果当前节点为空说明该商品还没有加入关系树木中
            if currentNode is None:
                currentNode = FpNode(code, parent)
                parent.child[code] = currentNode
                codeItems = self.fpNodeDict.get(code, [])
                codeItems.append(currentNode)
                self.fpNodeDict.setdefault(code, codeItems)
                parent = currentNode
                continue
                # 当前节点已经存在，直接对节点数量加一
            currentNode.add_count()
            parent = currentNode
        return self

    def build_tree(self, transaction):
        """
        根据所有订单构建树
        """
        for item in transaction:
            self.insert_node(item)
        return self

    def condition_merge(self, conditionList):
        """
        通过递归方式对同一条链路上的商品合并，获取其上所有可组合的频繁项集
        :param conditionList:链路上的所有商品
        :return: [(集合:count),(集合:count)]
        conditionList 中只有一个元素时候 则表示已经把所有情况组合完成
        """
        if len(conditionList) <= 1:
            return conditionList
        mergedConditionList = []
        for conditionItemIndex in range(len(conditionList)):
            conditionItem = conditionList[conditionItemIndex]
            groupConditionList = []
            for item in conditionList[conditionItemIndex+1:]:
                itemSet = set(item[0])
                itemSet.update(conditionItem[0])
                groupConditionList.append((itemSet, conditionItem[1]))
            # 对于同一层采用递归调用方式如
            #         A B C D E
            #           合成 AB AC AD AE后把它当作一个整体 做输入条件
            if len(groupConditionList) == 0:
                continue
            mergedConditionList.extend(self.condition_merge(groupConditionList))
        conditionList.extend(mergedConditionList)
        return conditionList

    def condition_frequent_item(self, conditionNodeList, comdition_item_count,min_threshold = 0):
        """
        对同一个商品的所有路径进行组合，得到条件模式基的频繁项集以及对应频次，并把各个路径的相同的平凡项集合并为同一个，数量叠加一起
        :param conditionNodeList: 同一个商品的所有路径列表
        :param min_threshold: 最小阈值
        """
        allConditionNodeList = []
        # 对不同路径上的商品进行组合，得到频繁项集
        for node in conditionNodeList:
            count = node.count
            parent = node.parent
            conditionList = []
            while parent.code is not None:
                code = parent.code
                conditionList.append(({code}, count))
                parent = parent.parent
            # 对 conditionList 进行自由组合 先从k=2开始组合
            allConditionNodeList.extend(self.condition_merge(conditionList))
        item_dict = {}
        for item in allConditionNodeList:
            item_tuple = tuple(item[0]);
            count = item_dict.pop(item_tuple, 0) + item[1]
            item_dict.setdefault(item_tuple, count)

        return [(k,v) for k, v in item_dict.items() if v >= min_threshold]

    def frequent_item_sets(self, items_list, min_sup=None):
        """
         获取条件基的频繁项集
         从排序最后的叶子作为条件模式基 向上获取条件频繁项集,
         然后根据最小支持度预集合得到最终频繁项集
        """
        minCount = min_sup * self.transaction_count
        for item in reversed(items_list):
            result = self.condition_frequent_item(self.fpNodeDict.get(item[0]),item[1], minCount)
            if len(result) > 0:
                print("条件模式基商品为：{},对应的可推荐的{}".format(item[0],result))

        return self


if __name__ == "__main__":
    is_find_min_support = False
    orgItemList = loadUseAbleData()
    itemsList = directItem(orgItemList, 0.02, 100)
    itemDict, orgItemList = sortedAndFiltter(itemsList, orgItemList)
    tree(itemDict, len(orgItemList)).build_tree(orgItemList).frequent_item_sets(itemsList, 0.02)


条件模式基商品为：soup,对应的可推荐的[(('mineral water',), 173)]
条件模式基商品为：cooking oil,对应的可推荐的[(('mineral water',), 151)]
条件模式基商品为：whole wheat rice,对应的可推荐的[(('mineral water',), 151)]
条件模式基商品为：chicken,对应的可推荐的[(('mineral water',), 171)]
条件模式基商品为：frozen smoothie,对应的可推荐的[(('mineral water',), 152)]
条件模式基商品为：olive oil,对应的可推荐的[(('spaghetti',), 172), (('mineral water',), 207)]
条件模式基商品为：tomatoes,对应的可推荐的[(('spaghetti',), 157), (('mineral water',), 183)]
条件模式基商品为：shrimp,对应的可推荐的[(('spaghetti',), 159), (('mineral water',), 177)]
条件模式基商品为：low fat yogurt,对应的可推荐的[(('mineral water',), 180)]
条件模式基商品为：cake,对应的可推荐的[(('mineral water',), 206)]
条件模式基商品为：burgers,对应的可推荐的[(('french fries',), 165), (('spaghetti',), 161), (('eggs',), 216), (('mineral water',), 183)]
条件模式基商品为：pancakes,对应的可推荐的[(('eggs',), 163), (('mineral water',), 253), (('french fries',), 151), (('spaghetti',), 189)]
条件模式基商品为：frozen vegetables,对应的可推荐的[(('spaghetti',), 209), (('mineral water',), 268), (('milk',), 177), (('chocolate',), 172), (('eggs',), 163)]
条件模式

结论：通过ml方法的结果中可以看到支持度高的，置信度不一定高，置信度高的 提升度不一定高，它的可以使用三个指标灵活切换，而fp-growth一次只能根据一个指标来计算，但是fp-growth在内存使用量上确实比ml方式的要小很多，因为在前期数据处理阶段就可以通过指标把大量数据给过滤掉，同时树结构也大大节省了内存空间，所以fp-growth适合处理大数据量处理，而小数据量的话使用ml会更好一些，指标丰富，灵活

疑问：超市中一个订单中只包含一个商品的应该占大多数，那么如果筛选出来的前一百个商品正好都是单订单商品，fp-growthß是否无法完成推荐呢？
  助教解答：首先可以在数据预处理部分对这部分数据进行处理，并且这部分数据其实也是很有统计意义的，可以结合用户特性进行分析，代表购买这部分商品的用户倾向于比较具有针对性地进行购物，此外，这部分数据也可以代表这类物品的全局属性，比如需求程度、物品使用或被购买的独立程度等等
